# Final Project
Logan Cooper

In [ ]:
import pandas as pd
import numpy as np
import gc
from tqdm.auto import tqdm
import statsmodels.formula.api as smf
from statsmodels.regression.linear_model import OLSResults
from typing import Tuple
import geopandas

## Data

### Data Import

#### NCDB Building Age Data (2000)

In [ ]:
ncdb_dtypes = {
    'tract_id': str,
    'workers_taking_transit': int,
    'prop_taking_transit': float,
    'built_1999_2000': int,
    'built_1995_1998': int,
    'built_1990_1994': int,
    'built_1980_1989': int,
    'built_1970_79': int,
    'built_1960_69': int,
    'built_1950_59': int,
    'built_1940_49': int,
    'built_1939_earlier': int
}
ncdb_data = pd.read_csv('./data/NCDB_2000.csv', dtype=ncdb_dtypes)
ncdb_data

In [ ]:
ncdb_data[ncdb_data['tract_id'].str.startswith('6')]

In [ ]:
ncdb_data['pub_trans_gt_10pct'] = 0
ncdb_data['pub_trans_gt_10pct'][ncdb_data['prop_taking_transit'] >= 0.1] = 1
ncdb_data

In [ ]:
ncdb_data[ncdb_data['tract_id'].str.startswith('6')]

In [ ]:
ncdb_data['pub_trans_gt_10pct'].value_counts()

In [ ]:
ncdb_data

#### Tract Level Data (2019)

In [ ]:
# import & rename cols
tract_data_2019 = pd.read_json('./data/tract_data_2019.json', dtype=False)
tract_data_2019.rename({
    'B19019_001E': 'median_income',
    'B01003_001E': 'population'
}, inplace=True, axis=1)

tract_data_2019['tract_id'] = tract_data_2019['state'] + tract_data_2019['county'] + tract_data_2019['tract']
tract_data_2019

In [ ]:
tract_data_2019['state'].value_counts().sort_index()

#### MSA-Level Income Data (2019)

In [ ]:
msa_data = pd.read_json('./data/msa_data.json')
msa_data.columns = ['name', 'median_income', 'msa_code']
msa_data['msa_code'] = msa_data['msa_code'].astype(str)
msa_data

In [ ]:
msa_data = msa_data[msa_data['name'].str.contains('Metro Area')]
msa_data

In [ ]:
# derived from an earlier notebook version where these MSA IDs turned up missing
# anything not in here is either in Puerto Rico or isn't an MSA in at least one period
changed_msa_ids = ['19380','29140','42260','31100','22460','11340','11300','14060','42060','26180','23020','39140']
changed_msa_names = ['Dayton-Kettering, OH Metro Area', 
                     'Lafayette-West Lafayette, IN Metro Area', 
                     'North Port-Sarasota-Bradenton, FL Metro Area',
                     'Los Angeles-Long Beach-Anaheim, CA Metro Area',
                     'Florence-Muscle Shoals, AL Metro Area',
                     'Greenville-Anderson, SC Metro Area',
                     'Indianapolis-Carmel-Anderson, IN Metro Area',
                     'Bloomington, IL Metro Area',
                     'Santa Maria-Santa Barbara, CA Metro Area',
                     'Urban Honolulu, HI Metro Area',
                     'Crestview-Fort Walton Beach-Destin, FL Metro Area',
                     'Prescott Valley-Prescott, AZ Metro Area']

msa_data.loc[msa_data['name'].isin(changed_msa_names), 'msa_code'] = changed_msa_ids
msa_data[msa_data['msa_code'].isin(changed_msa_ids)]


In [ ]:
msa_data.to_json('./data/msa_data.json')

#### MSA-Tract Crosswalk

In [ ]:
msa_lookup = pd.read_excel('./data/omb-cbsa-csa.xls')
msa_lookup = msa_lookup.iloc[2:].dropna(axis=1).iloc[:, [0, 2, -1]].reset_index(drop=True).dropna()
msa_lookup.columns = ['CBSA Code', 'Metropolitan/Micropolitan Statistical Area', 'FIPS Code']

for c in msa_lookup.columns:
    msa_lookup[c] = msa_lookup[c].astype(str)

msa_lookup

In [ ]:
msa_lookup['FIPS State Code'] = msa_lookup['FIPS Code'].str[:2]
msa_lookup['FIPS County Code'] = msa_lookup['FIPS Code'].str[2:]
msa_lookup.drop('FIPS Code', axis=1, inplace=True)
msa_lookup

In [ ]:
og_msas = msa_lookup[msa_lookup['Metropolitan/Micropolitan Statistical Area'] == 'Metropolitan Statistical Area']['CBSA Code'].value_counts().index
og_msas

In [ ]:
# msa_lookup = pd.read_excel('./data/msa_codes.xls', 
#                            converters={'CBSA Code': str, 
#                                        'CBSA Title': str, 
#                                        'Metropolitan/Micropolitan Statistical Area': str, 
#                                        'FIPS State Code': str,
#                                        'FIPS County Code': str}
#                            ).drop(range(1916,1920))

# msa_lookup

#### Conversion for 2000 => 2010

In [ ]:
tract_conversion = pd.read_csv('./data/us2010trf.txt')
tract_conversion

In [ ]:
tract_conversion.columns

In [ ]:
tract_conversion = tract_conversion[['GEOID00', 'GEOID10', 'AREALAND10']]
tract_conversion['GEOID00'] = tract_conversion['GEOID00'].astype(str)
tract_conversion['GEOID10'] = tract_conversion['GEOID10'].astype(str)
tract_conversion

In [ ]:
# AREALAND10 is in square meters, want it in square miles
tract_conversion['AREALAND10'] = tract_conversion['AREALAND10'] * 3.861e-7
tract_conversion = tract_conversion[tract_conversion['AREALAND10'] > 0]
tract_conversion

#### Tract-School District Crosswalk

In [ ]:
school_districts = pd.read_excel('./data/grf19_lea_tract.xlsx')
# school_districts.drop(['NAME_LEA19', 'COUNT', 'LANDAREA', 'WATERAREA'], axis=1, inplace=True)
school_districts['LEAID'] = school_districts['LEAID'].astype(str)
school_districts['TRACT'] = school_districts['TRACT'].astype(str)
school_districts

#### Wilson CBD

In [ ]:
wilson_cbd = pd.read_csv('./data/wilson_cbd.csv', index_col=False, dtype={'msa_id': str, 'wilson_cbd': str})
wilson_cbd

## Data Combination

#### Trim Tract Level Data to MSAs Only

In [ ]:
only_metros = msa_lookup[msa_lookup['Metropolitan/Micropolitan Statistical Area'] == 'Metropolitan Statistical Area']
msa_tracts = pd.merge(left=tract_data_2019, right=only_metros, left_on=['state', 'county'], right_on=['FIPS State Code', 'FIPS County Code'])
msa_tracts

In [ ]:
msa_tracts = msa_tracts[(msa_tracts['median_income'] >= 0) & (msa_tracts['population'] > 0)]
msa_tracts

In [ ]:
msa_tracts['state'].value_counts().sort_index()

In [ ]:
msa_tracts['CBSA Code'].value_counts()

In [ ]:
pr_msas = [el for el in og_msas if (el not in msa_tracts['CBSA Code'].value_counts().index)] #okay looks like these are all in PR
pr_msas

In [ ]:
msa_tracts.columns

In [ ]:
msa_tracts.drop(['Metropolitan/Micropolitan Statistical Area', 'FIPS State Code', 'FIPS County Code'], axis=1, inplace=True)
msa_tracts

In [ ]:
del only_metros
gc.collect()

#### Merge MSA-Level Data

In [ ]:
msa_tracts = pd.merge(left=msa_tracts, right=msa_data, left_on='CBSA Code', right_on='msa_code', suffixes=('_tract', '_msa'))
msa_tracts['income'] = msa_tracts['median_income_tract'] / msa_tracts['median_income_msa']
msa_tracts.drop(['CBSA Code', 'median_income_tract', 'median_income_msa'], axis=1, inplace=True)
msa_tracts

In [ ]:
[el for el in og_msas if (el not in msa_tracts['msa_code'].value_counts().index) and (el not in pr_msas)]

In [ ]:
msa_tracts['state'].value_counts().sort_index()

In [ ]:
# msa_tracts[msa_tracts['tract_id'].str.startswith('0')]['tract_id'] = msa_tracts[msa_tracts['tract_id'].str.startswith('0')]['tract_id'].str[1:]

In [ ]:
del msa_data
gc.collect()

#### Convert 2000-Tracts and Merge

In [ ]:
ages_2019 = pd.merge(left=ncdb_data, right=tract_conversion, left_on='tract_id', right_on='GEOID00')
ages_2019.drop(['tract_id', 'workers_taking_transit', 'prop_taking_transit'], axis=1, inplace=True)
ages_2019.rename({'AREALAND10': 'area', 'GEOID10': 'tract_id_2010', 'GEOID00': 'tract_id_2000'}, inplace=True, axis=1)

ages_2019

In [ ]:
non_aggregated_tract_data = ages_2019[['tract_id_2010', 'area']].drop_duplicates()
non_aggregated_tract_data

In [ ]:
summed_ages = ages_2019.groupby('tract_id_2010').sum(numeric_only=True).drop(['area'], axis=1).reset_index()
summed_ages

In [ ]:
summed_ages['pub_trans_gt_10pct'] = (summed_ages['pub_trans_gt_10pct'] >= 1).astype(int)
summed_ages

In [ ]:
summed_ages.describe()

In [ ]:
msa_tracts['tract_id'] = msa_tracts['tract_id'].str.removeprefix('0')
msa_tracts['tract_id']

In [ ]:
ages_2019 = pd.merge(left=summed_ages, right=non_aggregated_tract_data)
ages_2019

In [ ]:
ages_2019.describe()

In [ ]:
msa_tracts = pd.merge(left=msa_tracts, right=ages_2019, left_on='tract_id', right_on='tract_id_2010')
msa_tracts.drop(['tract_id'], axis=1, inplace=True)

msa_tracts = msa_tracts[(msa_tracts['population'] >= 1) & (msa_tracts['income'] > 0) & (msa_tracts['area'] > 0)] # drop weird tracts

msa_tracts['pop_density'] = msa_tracts['population'] / msa_tracts['area']
msa_tracts

In [ ]:
msa_tracts['state'].value_counts().sort_index()

In [ ]:
del ages_2019
gc.collect()

#### Add School District

In [ ]:
msa_tracts = pd.merge(left=msa_tracts, right=school_districts, left_on='tract_id_2010', right_on='TRACT')
msa_tracts.drop(['NAME_LEA19', 'TRACT', 'COUNT', 'LANDAREA', 'WATERAREA'], axis=1, inplace=True)
msa_tracts

In [ ]:
del school_districts
gc.collect()

### Add Wilson CBD

In [ ]:
wilson_cbd

In [ ]:
msa_tracts = pd.merge(left=msa_tracts, right=wilson_cbd, left_on='msa_code', right_on='msa_id')
msa_tracts.drop('msa_id', axis=1, inplace=True)
msa_tracts['wilson_cbd'] = msa_tracts['wilson_cbd'].str.removeprefix('0')
msa_tracts

In [ ]:
msa_tracts

In [ ]:
msa_tracts.to_csv('./data/msa_tracts.csv', index=False)

### Calculating Distances

#### Finding Central Business District

In [ ]:
msa_tracts = pd.read_csv('./data/msa_tracts.csv', index_col=False)
msa_tracts.rename({'msa_code_tract': 'msa_code'}, axis=1, inplace=True)
msa_tracts.drop(['area', 'population'], inplace=True, axis=1)
msa_tracts['wilson_cbd'] = msa_tracts['wilson_cbd'].astype(str)
msa_tracts

In [ ]:
msa_tracts['msa_code'].value_counts()

In [ ]:
msa_tracts['state'].value_counts().sort_index()

In [ ]:
cbd_candidates = msa_tracts[['msa_code', 'tract_id_2010', 'pop_density']]
cbds = cbd_candidates.groupby('msa_code').max()
cbds.drop('pop_density', axis=1, inplace=True)
cbds.reset_index(inplace=True)
cbds

In [ ]:
msa_tracts = pd.merge(left=msa_tracts, right=cbds, left_on='msa_code', right_on='msa_code', suffixes=('', 'cbd'))
msa_tracts.rename({'tract_id_2010cbd': 'br_cbd'}, inplace=True, axis=1)
msa_tracts['br_cbd'] = msa_tracts['br_cbd'].astype(str)
msa_tracts['tract_id_2010'] = msa_tracts['tract_id_2010'].astype(str)
msa_tracts

In [ ]:
msa_tracts['br_cbd'].str.len().value_counts()

In [ ]:
msa_tracts['tract_id_2010'].str.len().value_counts()

#### Calculating Distances

In [ ]:
def regularize_house_ages(df: pd.DataFrame) -> pd.DataFrame:
    age_cats = ['built_1999_2000',
       'built_1995_1998', 'built_1990_1994', 'built_1980_1989',
       'built_1970_1979', 'built_1960_1969', 'built_1950_1959',
       'built_1940_1949', 'built_1939_earlier']

    build_totals = df[age_cats].sum(axis=1)

    df[age_cats] = df[age_cats].div(build_totals, axis=0)
    return df
    

def get_distances_for_cbd(df: pd.DataFrame, cbd_measure: str, chunksize: int) -> pd.DataFrame:
    print(f'Starting for {cbd_measure}...')
    msa_tracts_with_dist = None
    tract_distances = pd.read_csv('./data/sf12010tractdistance50miles.csv', dtype={'county1': str,'tract1': str, 'county2': str,'tract2': str}, chunksize=chunksize)
    for chunk in tqdm(tract_distances):
        chunk['tid1'] = (chunk['county1'] + chunk['tract1']).str.removeprefix('0')
        chunk.drop(['county1', 'tract1'], axis=1, inplace=True)
        chunk['tid2'] = (chunk['county2'] + chunk['tract2']).str.removeprefix('0')
        chunk.drop(['county2', 'tract2'], axis=1, inplace=True)
        chunk.rename({'mi_to_tract': 'distance'}, axis=1, inplace=True)

        m = pd.merge(left=df, right=chunk, left_on=[cbd_measure, 'tract_id_2010'], right_on=['tid1', 'tid2'])
        if msa_tracts_with_dist is None:
            msa_tracts_with_dist = m
        else:
            msa_tracts_with_dist = pd.concat((m, msa_tracts_with_dist))

    msa_tracts_with_dist.drop(['tid1', 'tid2'], axis=1, inplace=True)
    msa_tracts_with_dist = msa_tracts_with_dist[msa_tracts_with_dist['distance'] <= 40]

    cbd_tracts = df[df['tract_id_2010'] == df[cbd_measure]]
    cbd_tracts['distance'] = 0
    msa_tracts_with_dist = pd.concat((msa_tracts_with_dist, cbd_tracts))
    msa_tracts_with_dist.drop(['state', 'county', 'tract'], axis=1, inplace=True)

    return regularize_house_ages(msa_tracts_with_dist)

In [ ]:
msa_tracts_with_dist_br = get_distances_for_cbd(msa_tracts, 'br_cbd', chunksize=200000)
msa_tracts_with_dist_br

In [ ]:
msa_tracts_with_dist_br['msa_code'].value_counts()

In [ ]:
msa_tracts_with_dist_br.to_csv('./data/msa_tracts_dist_br.csv', index=False)

In [ ]:
msa_tracts_with_dist_wilson = get_distances_for_cbd(msa_tracts, 'wilson_cbd', chunksize=200000)
msa_tracts_with_dist_wilson

In [ ]:
msa_tracts_with_dist_wilson['msa_code'].value_counts()

In [ ]:
msa_tracts_with_dist_wilson.to_csv('./data/msa_tracts_dist_wilson.csv', index=False)

## Modelling

The smallest of these models runs OLS with ~73,000 data points and 300 fixed effects. Therefore, I wasn't able to run most of them locally. Instead, I ran the models on the Duke Economics Computing Cluster and downloaded the saved models. The process for this can be seen in `reg.py`. Note that the results below omit the several thousand fixed effects.

In [ ]:
br = pd.read_csv('./data/msa_tracts_dist_br.csv', index_col=False, dtype={'msa_code': str, 'LEAID': str, 'pub_trans_gt_10pct': int}).dropna()
wilson = pd.read_csv('./data/msa_tracts_dist_wilson.csv', index_col=False, dtype={'msa_code': str, 'LEAID': str, 'pub_trans_gt_10pct': int}).dropna()

In [ ]:
br

In [ ]:
wilson

In [ ]:
br.describe()

In [ ]:
wilson.describe()

In [ ]:
import re
fixed_effects = re.compile(r'C\([A-Za-z_]+\).+\n')

def print_model_output(model_type: str, model_num: int, model: str) -> str:
    with open(f'./models/{model}/model-{model_type}-{model_num}-summary.txt', 'r') as f:
        print(fixed_effects.sub('', f.read()))

### Brueckner-Rosenthal-Distance Results

In [ ]:
print_model_output('br', 1, 'WLS')

In [ ]:
print_model_output('br', 2, 'WLS')

In [ ]:
print_model_output('br', 3, 'WLS')

In [ ]:
print_model_output('br', 4, 'WLS')

In [ ]:
print_model_output('br', 5, 'WLS')

### Wilson-Distance Results

In [ ]:
print_model_output('wilson', 1, 'WLS')

In [ ]:
print_model_output('wilson', 2, 'WLS')

In [ ]:
print_model_output('wilson', 3, 'WLS')

In [ ]:
print_model_output('wilson', 4, 'WLS')

In [ ]:
print_model_output('wilson', 5, 'WLS')

## Graphing

In [ ]:
from matplotlib import pyplot as plt

br.plot(kind='scatter', x='distance', y='income', xlabel='BR-Distance', ylabel='Tract Income', title='Brueckner-Rosenthal')
wilson.plot(kind='scatter', x='distance', y='income', xlabel='Wilson-Distance', ylabel='Tract Income', title='Wilson')

In [ ]:
plt.hexbin(x=br['distance'], y=br['income'], gridsize=(40, 40))
plt.xlabel('BR-Distance')
plt.ylabel('Tract Income')
plt.title('Brueckner-Rosenthal')

In [ ]:
plt.hexbin(x=wilson['distance'], y=wilson['income'], gridsize=(40, 40))
plt.xlabel('Wilson-Distance')
plt.ylabel('Tract Income')
plt.title('Wilson')

In [ ]:
def add_tract_counts(df: pd.DataFrame) -> pd.DataFrame:
    tract_counts =df.groupby('msa_code').count()['income']
    tract_counts = tract_counts.reset_index().rename({'income': 'tract_counts'}, axis=1)
    return pd.merge(right=df, left=tract_counts, left_on='msa_code', right_on='msa_code')

def discretize_distances(df: pd.DataFrame) -> pd.DataFrame:
    df['dist_bin'] = df['distance'].round()
    return df

def split_df_by_tract_counts(df: pd.DataFrame) -> Tuple[pd.DataFrame]:
    under_100 = df[df['tract_counts'] < 100]
    btw_100_500 = df[(df['tract_counts'] >= 100) & (df['tract_counts'] < 500)]
    btw_500_1000 = df[(df['tract_counts'] >= 500) & (df['tract_counts'] < 1000)]
    larger_1000 = df[df['tract_counts'] >= 1000]

    return (under_100, btw_100_500, btw_500_1000, larger_1000)

def plot_relative_income(df: pd.DataFrame, title: str, dist: str) -> None:
    df_dist_inc = df.groupby('dist_bin').mean()['income']
    plt.plot((df_dist_inc - df_dist_inc[0]) / df_dist_inc[0])
    plt.xlabel(f'{dist}-Distance from CBD (mi)')
    plt.ylabel('Median Income Compared to Mile Zero')
    plt.title(title)

def plot_housing_dist(df: pd.DataFrame) -> None:
    df.groupby('dist_bin').mean()[
    ['built_1999_2000', 
        'built_1995_1998', 
        'built_1990_1994', 
        'built_1980_1989', 
        'built_1970_1979', 
        'built_1960_1969', 
        'built_1950_1959', 
        'built_1940_1949', 
        'built_1939_earlier']
    ].plot(xlabel='Distance from CBD (mi)', ylabel='Mean Share of Housing Age')

def process_dataframe(df: pd.DataFrame) -> Tuple[pd.DataFrame]:
    return split_df_by_tract_counts(discretize_distances(add_tract_counts(df)))

In [ ]:
under_100_br, btw_100_500_br, btw_500_1000_br, larger_1000_br = process_dataframe(br)
under_100_wilson, btw_100_500_wilson, btw_500_1000_wilson, larger_1000_wilson = process_dataframe(wilson)

In [ ]:
plot_relative_income(under_100_br, 'Under 100 Tracts', 'BR')

In [ ]:
plot_relative_income(btw_100_500_br, '100-499 Tracts', 'BR')

In [ ]:
plot_relative_income(btw_500_1000_br, '500-999 Tracts', 'BR')

In [ ]:
plot_relative_income(larger_1000_br, '1000 or More Tracts', 'BR')

In [ ]:
plot_relative_income(under_100_wilson, 'Under 100 Tracts', 'Wilson')

In [ ]:
plot_relative_income(btw_100_500_wilson, '100-499 Tracts', 'Wilson')

In [ ]:
plot_relative_income(btw_500_1000_wilson, '500-999 Tracts', 'Wilson')

In [ ]:
plot_relative_income(larger_1000_wilson, '1000 or More Tracts', 'Wilson')

In [ ]:
plot_relative_income(discretize_distances(wilson), 'All Cities', 'Wilson')

## Mapping

In [ ]:
def geofy_data(df: pd.DataFrame) -> geopandas.GeoDataFrame:
    ca_geo = geopandas.read_file('./data/CA_shapefile/tl_2010_06_tract00.shp')
    ca_geo = ca_geo[['CTIDFP00', 'geometry']]

    ca_geo['CTIDFP00'] = ca_geo['CTIDFP00'].astype(str).str.removeprefix('0')
    df['tract_id_2010'] = df['tract_id_2010'].astype(str)

    ca_geo = pd.merge(left=ca_geo, right=df, left_on='CTIDFP00', right_on='tract_id_2010')
    return ca_geo.dropna()